Load the required libraries

In [ ]:
import gspread as gs
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import pickle

Establish a connection with Google Sheets

In [ ]:
gc = gs.service_account(filename='spreadsheetxtern.json')
sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1y_g37nbDgHC-zm1y_WGfThBt5wipMxnFSDcSZT6AQq4/edit#gid=651709921')
ws = sh.worksheet('Xtern_TrainData')
df = pd.DataFrame(ws.get_all_records())

Refine the dataset

In [ ]:
X = df.drop('Order', axis=1)
y = df['Order']
enc = OrdinalEncoder()
X = pd.DataFrame(enc.fit_transform(X), columns=X.columns)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Construct the model with the data

In [ ]:
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

Save the model, scaler, and encoder

In [ ]:
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
with open('encoder.pkl', 'wb') as f:
    pickle.dump(enc, f)

Load the model, scaler, and encoder

In [ ]:
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)
with open('encoder.pkl', 'rb') as f:
    enc = pickle.load(f)

Seek input from the user and make anticipations

In [ ]:
year = input("Enter year of study (e.g., 'Year 2'): ")
major = input("Enter major (e.g., 'Mathematics'): ")
university = input("Enter university (e.g., 'Indiana State University'): ")
time = int(input("Enter time (an integer from 0 to 24): "))
new_data = pd.DataFrame([[year, major, university, time]], columns=['Year', 'Major', 'University', 'Time'])
new_data_encoded = pd.DataFrame(enc.transform(new_data), columns=new_data.columns)
new_data_scaled = scaler.transform(new_data_encoded)
predictions_proba = model.predict_proba(new_data_scaled)

for i in range(len(model.classes_)):
    print(f"Probability of ordering {model.classes_[i]}: {predictions_proba[0][i]*100}%")